In [6]:
import pandas as pd
import csv

In [7]:
school_data_in = "Resources/schools_complete.csv"
student_data_in = "Resources/students_complete.csv"

In [8]:
# Read School and Student Data
school_data = pd.read_csv(school_data_in)
student_data = pd.read_csv(student_data_in)

In [10]:
school_data_full = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
data_df = pd.DataFrame(school_data_full)

data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary Info

Included Summary info -
Total number of unique schools
Total students
Total budget
Average math score
Average reading score
% passing math (the percentage of students who passed math)
% passing reading (the percentage of students who passed reading)
% overall passing (the percentage of students who passed math AND reading)

In [21]:
#school and student counts
school_count = data_df['school_name'].nunique()
student_count = data_df['Student ID'].nunique()
#budget total
budget = data_df['budget'].unique()
total_budget = sum(budget)

#average math and reading scores
avg_math_score = data_df['math_score'].mean()
avg_reading_score = data_df['reading_score'].mean()

#% passing math (the percentage of students who passed math)
#count of students passing math (>=70)
pass_math_count = school_data_full[(school_data_full["math_score"] >= 70)].count()["student_name"]

#convert count to percentage
pass_math_percentage = pass_math_count / float(student_count) * 100
#% passing reading (the percentage of students who passed reading)
pass_reading_count = school_data_full[(school_data_full["reading_score"] >= 70)].count()["student_name"]
pass_reading_percentage = pass_reading_count / float(student_count) * 100

#% overall passing (the percentage of students who passed math AND reading)
pass_math_reading_count = school_data_full[(school_data_full["math_score"] >= 70) & (school_data_full["reading_score"] >= 70)].count()["student_name"]
overall_pass_rate = pass_math_reading_count /  float(student_count) * 100

print(f"School count = {school_count}")
print(f"Student count = {student_count}")
print(f"Total budget = {total_budget}")
print(f"Avg math score = {avg_math_score}")
print(f"Avg reading score = {avg_reading_score}")
print(f"Math passing rate = {pass_math_percentage}%")
print(f"Reading passing rate = {pass_reading_percentage}%")
print(f"Overall Math/Reading passing rate = {overall_pass_rate}%")


School count = 15
Student count = 39170
Total budget = 24649428
Avg math score = 78.98537145774827
Avg reading score = 81.87784018381414
Math passing rate = 74.9808526933878%
Reading passing rate = 85.80546336482001%
Overall Math/Reading passing rate = 65.17232575950983%
